# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
tf.__version__ 

'1.9.0'

## Env setup

In [5]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [6]:
from utils import label_map_util

from utils import visualization_utils as vis_util

C:\Users\midas\Anaconda3\lib\site-packages\matplotlib\__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [7]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

In [8]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [9]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [10]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [11]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [102]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 8) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [103]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [121]:
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  if output_dict['detection_scores'][0]*100 > 80:
        print("hi")
#       vis_util.visualize_boxes_and_labels_on_image_array(
#           image_np,
#           output_dict['detection_boxes'],
#           output_dict['detection_classes'],
#           output_dict['detection_scores'],
#           category_index,
#           instance_masks=output_dict.get('detection_masks'),
#           use_normalized_coordinates=True,
#           line_thickness=8)
#       plt.figure(figsize=IMAGE_SIZE)
#       plt.imshow(image_np)

hi
hi
hi
hi
hi
hi
hi


In [122]:
output_dict['detection_scores']

array([0.81022793, 0.75924635, 0.6349491 , 0.631374  , 0.44451538,
       0.41424653, 0.33253625, 0.32778132, 0.28748158, 0.26579508,
       0.22938915, 0.22152407, 0.18941027, 0.17206055, 0.16651578,
       0.16520852, 0.14966042, 0.14625923, 0.13942283, 0.13021946,
       0.1207901 , 0.11912149, 0.1162246 , 0.11219613, 0.11040147,
       0.10834774, 0.10557394, 0.10536908, 0.10416987, 0.10038005,
       0.10022218, 0.09946947, 0.09792275, 0.09545403, 0.09351873,
       0.09253298, 0.09235419, 0.09093217, 0.09090099, 0.08993652,
       0.08982867, 0.08900051, 0.08810627, 0.08799742, 0.0872272 ,
       0.0865031 , 0.08378388, 0.0833632 , 0.08262639, 0.08261248,
       0.08246815, 0.08109002, 0.07993674, 0.07951298, 0.07861707,
       0.07704619, 0.07690889, 0.07599699, 0.07569487, 0.07509055,
       0.07436669, 0.07376361, 0.07207183, 0.07188773, 0.07156765,
       0.07122198, 0.06908181, 0.06878553, 0.06848809, 0.06785093,
       0.06781529, 0.0676493 , 0.06759986, 0.06728604, 0.06697

In [24]:
image_path


'test_images\\image2.jpg'

In [54]:
output_dict.keys()

dict_keys(['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes'])

In [95]:
output_dict['detection_classes'].shape

(100,)

In [58]:
ops = tf.get_default_graph().get_operations()
print({output.name for op in ops for output in op.outputs})

set()


In [94]:
print(
category_index[1]['name'],
category_index[1]['id']
)

Muzafar 1


In [93]:
category_index[1]['name']='Muzafar'

In [96]:
category_index

{1: {'id': 1, 'name': 'Muzafar'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suit

In [97]:
output_dict['detection_scores']

{'detection_boxes': array([[0.13445586, 0.37801963, 0.9802    , 0.7312389 ],
        [0.08996972, 0.41869068, 0.98888016, 0.9858142 ],
        [0.24390861, 0.5181731 , 0.3512024 , 0.581203  ],
        [0.7501227 , 0.40462396, 0.94094056, 0.68308973],
        [0.65430033, 0.38531855, 0.94611144, 0.70436835],
        [0.79053   , 0.3818318 , 0.9398636 , 0.65159607],
        [0.744246  , 0.5785982 , 0.9912919 , 0.70941216],
        [0.42768094, 0.52862686, 0.93396354, 0.7359455 ],
        [0.26889002, 0.5101139 , 0.3000834 , 0.53715986],
        [0.24068162, 0.48617828, 0.9947034 , 0.8346486 ],
        [0.18596202, 0.24748908, 0.85383433, 0.6880511 ],
        [0.5007683 , 0.4299774 , 0.9958985 , 0.66819036],
        [0.4083426 , 0.35366553, 0.98368776, 0.67582613],
        [0.682845  , 0.4151153 , 0.8770666 , 0.68449825],
        [0.7454468 , 0.43269375, 0.92821944, 0.68953884],
        [0.7395538 , 0.5265646 , 0.9650245 , 0.69399524],
        [0.26874554, 0.52517605, 0.38425738, 0.586663

In [101]:
output_dict['detection_scores'][0]*100>70

True

In [108]:
subgroup=['a','a','b','b','b']
a={'First':subgroup,'First':subgroup}



In [ ]:
Group  SubGroup  Target
First           a               1
First         a              2
First         b       1
First          b         2
First          b          3
Second c          1
Second  d          1
Third  e          1
Third  f          1
Third  f          2